In [60]:
# pip3 install httplib2 --upgrade
# pip install beautifulsoup4

import httplib2
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd

# useful links
# http://technology.chtsai.org/charfreq/sorted.html
# https://www.learnchineseez.com/read-write/simplified/

#
# Download Chinese Characters Sorted By frequency of use
#

#
# -------------------------
# Column  Description
# -------------------------
# 1-2     Character
# 8-14    Frequency
# 17-18   Number of Strokes
# -------------------------
#

# utf-8
# http://technology.chtsai.org/charfreq/sorted.html

# class main
http = httplib2.Http()

url = 'http://technology.chtsai.org/charfreq/sorted.html'
character_frequency_output_fname = "character_frequency_stroke.csv"
status, response = http.request(url)
soup = BeautifulSoup(response)

main_div = soup.find_all("div", class_="main")[0]
main_string = str(main_div.contents[1])

lines = main_string.splitlines()[9:-1]
character_frequency_dict_list = []
for l in lines:
    
    character = l[0:2].split(' ')[0]
    frequency = int(l[6:13])
    number_of_strokes = int(l[16:18])
    
    character = {
        'character' : character,
        'frequency' : frequency,
        'number_of_strokes' : number_of_strokes
    }
    character_frequency_dict_list.append(character)
    
characters_df = pd.DataFrame(character_frequency_dict_list)

# utf-8 encoding
characters_df.to_csv(character_frequency_output_fname, encoding='utf-8')

testing_df = pd.read_csv(character_frequency_output_fname)

print(testing_df)

       Unnamed: 0 character  frequency  number_of_strokes
0               0         的    6538132                  8
1               1         是    3200626                  9
2               2         不    2831612                  4
3               3         我    2584497                  7
4               4         一    2542556                  1
5               5         有    2289333                  6
6               6         大    1891383                  3
7               7         在    1715554                  6
8               8         人    1598855                  2
9               9         了    1507218                  2
10             10         中    1322363                  4
11             11         到    1310850                  8
12             12         資    1115608                  3
13             13         要    1034142                  9
14             14         以     994958                  5
15             15         可     992842                  5
16            

In [ ]:
#
# cedict import cedict file example
#

# pip install pycedict
# download 'cedict_1_0_ts_utf-8_mdbg.txt' from
#     https://www.mdbg.net/chinese/dictionary?page=cc-cedict

import cedict
infile = open('cedict_1_0_ts_utf-8_mdbg.txt')


for ch, chs, pinyin, defs, variants, mw in cedict.iter_cedict(infile):
    print(chs, pinyin, defs) # or probably load this info into your database
    
# pinyinize depinyinize example
print(cedict.pinyinize('ni3hao3'))
print(cedict.depinyinize('nǐhǎo'))

In [83]:
#
# create cedict pandas dataframe
# 

# load this info into csv
infile = open('cedict_1_0_ts_utf-8_mdbg.txt')
cedict_df_csv_file = 'cedict_dataframes/cedict_df.csv'

cedict_list = []
for ch, chs, pinyin, defs, variants, mw in cedict.iter_cedict(infile):
    cedict_i = {
        'ch' : ch,
        'chs' : chs,
        'pinyin' : pinyin,
        'defs' : defs,
        'variants' : variants,
        'mw' : mw
    }
    cedict_list.append(cedict_i)
    #print(chs, pinyin, defs) 

cedict_df = pd.DataFrame(cedict_list)

# utf-8 encoding
cedict_df.to_csv(cedict_df_csv_file, encoding='utf-8')

In [84]:
# load dfs
import cedict

top_frequency_df = pd.read_csv(character_frequency_output_fname)

cedict_df = pd.read_csv(cedict_df_csv_file)

In [98]:
def search_cedict_character(character):
    top_character_info_df = cedict_df.loc[cedict_df['ch'] == character]
    definitions = top_character_info_df['defs'].tolist()
    pinyin = top_character_info_df['pinyin'].tolist()
    variants = top_character_info_df['variants'].tolist()
    
    results = {
        'defintions' : definitions,
        'pinyin' : pinyin,
        'variants' : variants
    }
    return results
 

In [99]:
# example display information on top character

top_character = top_frequency_df['character'].iloc[0]


print( search_cedict_character(top_character) )


{'defintions': ['[\'of\', "~\'s (possessive particle)", \'(used after an attribute)\', \'(used to form a nominal expression)\', \'(used at the end of a declarative sentence for emphasis)\']', "['see 的士[di1 shi4]']", "['really and truly']", "['aim', 'clear']"], 'pinyin': ['de5', 'di1', 'di2', 'di4'], 'variants': ['[]', '[]', '[]', '[]']}
